In [23]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from pyspark.sql import SparkSession

In [24]:
# Khởi tạo SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MongoDB to Spark") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.2") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/dbmycrawler") \
    .getOrCreate()

In [25]:
#doc du lieu tu mongo
df=spark.read.format("mongo").option("collection","tblcafeland").load()
#hien du lieu
df.show() 

+--------------------+----------+------------------+--------------------+--------------------+--------------------+-----------+--------------------+-----------------+--------+--------------------+--------------------+
|          Chủ đầu tư|Diện tích:|        Loại dự án|               Mô tả|           Ngày đăng|           Thành phố| Trạng thái|           Tên dự án|  Tổng vốn đầu tư|Xếp hạng|                 _id|               Đường|
+--------------------+----------+------------------+--------------------+--------------------+--------------------+-----------+--------------------+-----------------+--------+--------------------+--------------------+
|Công Ty Cổ Phần B...|   8.200m2|     Đất Nền Dự Án|The Long Eyes là ...|Cập nhật: 12/09/2...|           Trảng Bom|Đang mở bán|The Long Eyes: Dự...|          unknown|     5.0|{66f8f9c98ac17558...|   đường Sông Thao 5|
|Công ty Cổ phần Đ...|33.540,7m2|   Căn hộ chung cư|Golden City là că...|Cập nhật: 25/09/2...|        TP. Tây Ninh|Đang mở bán|G

In [26]:
#Xem kiểu dl các cột
df.printSchema()

root
 |-- Chủ đầu tư: string (nullable = true)
 |-- Diện tích:: string (nullable = true)
 |-- Loại dự án: string (nullable = true)
 |-- Mô tả: string (nullable = true)
 |-- Ngày đăng: string (nullable = true)
 |-- Thành phố: string (nullable = true)
 |-- Trạng thái: string (nullable = true)
 |-- Tên dự án: string (nullable = true)
 |-- Tổng vốn đầu tư: string (nullable = true)
 |-- Xếp hạng: string (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- Đường: string (nullable = true)



In [27]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Khởi tạo SparkSession
spark = SparkSession.builder.appName("Example").getOrCreate()



# Thay thế giá trị 'unknown' và chuỗi rỗng trong tất cả các cột
for column in df.columns:
    if column in df.columns:  # Kiểm tra xem cột có tồn tại trong DataFrame không
        try:
            df = df.withColumn(column, 
                               F.when(F.col(column) == 'unknown', 'chưa cập nhật')
                                .when(F.col(column) == '', 'chưa cập nhật')
                                .otherwise(F.col(column)))
            print(f"Processed column: {column}")  # In ra cột đã xử lý
        except Exception as e:
            print(f"Error processing column {column}: {e}")  # In ra lỗi nếu có

# Kiểm tra dữ liệu sau khi thay thế
df.show()


Processed column: Chủ đầu tư
Processed column: Diện tích:
Processed column: Loại dự án
Processed column: Mô tả
Processed column: Ngày đăng
Processed column: Thành phố
Processed column: Trạng thái
Processed column: Tên dự án
Processed column: Tổng vốn đầu tư
Processed column: Xếp hạng
Error processing column _id: [DATATYPE_MISMATCH.BINARY_OP_DIFF_TYPES] Cannot resolve "(_id = unknown)" due to data type mismatch: the left and right operands of the binary operator have incompatible types ("STRUCT<oid: STRING>" and "STRING").;
'Project [Chủ đầu tư#1000, Diện tích:#1013, Loại dự án#1026, Mô tả#1039, Ngày đăng#1052, Thành phố#1065, Trạng thái#1078, Tên dự án#1091, Tổng vốn đầu tư#1104, Xếp hạng#1117, CASE WHEN (_id#925 = unknown) THEN chưa cập nhật WHEN (_id#925 = ) THEN chưa cập nhật ELSE _id#925 END AS _id#1130, Đường#926]
+- Project [Chủ đầu tư#1000, Diện tích:#1013, Loại dự án#1026, Mô tả#1039, Ngày đăng#1052, Thành phố#1065, Trạng thái#1078, Tên dự án#1091, Tổng vốn đầu tư#1104, CASE WH

In [28]:
#Xem kiểu dl các cột
df.show()

+--------------------+-------------+------------------+--------------------+--------------------+--------------------+-----------+--------------------+-----------------+--------+--------------------+--------------------+
|          Chủ đầu tư|   Diện tích:|        Loại dự án|               Mô tả|           Ngày đăng|           Thành phố| Trạng thái|           Tên dự án|  Tổng vốn đầu tư|Xếp hạng|                 _id|               Đường|
+--------------------+-------------+------------------+--------------------+--------------------+--------------------+-----------+--------------------+-----------------+--------+--------------------+--------------------+
|Công Ty Cổ Phần B...|      8.200m2|     Đất Nền Dự Án|The Long Eyes là ...|Cập nhật: 12/09/2...|           Trảng Bom|Đang mở bán|The Long Eyes: Dự...|    chưa cập nhật|     5.0|{66f8f9c98ac17558...|   đường Sông Thao 5|
|Công ty Cổ phần Đ...|   33.540,7m2|   Căn hộ chung cư|Golden City là că...|Cập nhật: 25/09/2...|        TP. Tây Nin

In [29]:
from pyspark.sql.functions import col, regexp_replace
# Loại bỏ chuỗi 'Cập nhật:' trong cột 'Ngày đăng'
df = df.withColumn("Ngày đăng", regexp_replace(col("Ngày đăng"), "Cập nhật:", ""))

In [30]:
#Xem kiểu dl các cột
df.show()

+--------------------+-------------+------------------+--------------------+--------------------+--------------------+-----------+--------------------+-----------------+--------+--------------------+--------------------+
|          Chủ đầu tư|   Diện tích:|        Loại dự án|               Mô tả|           Ngày đăng|           Thành phố| Trạng thái|           Tên dự án|  Tổng vốn đầu tư|Xếp hạng|                 _id|               Đường|
+--------------------+-------------+------------------+--------------------+--------------------+--------------------+-----------+--------------------+-----------------+--------+--------------------+--------------------+
|Công Ty Cổ Phần B...|      8.200m2|     Đất Nền Dự Án|The Long Eyes là ...|  12/09/2024 8:49 AM|           Trảng Bom|Đang mở bán|The Long Eyes: Dự...|    chưa cập nhật|     5.0|{66f8f9c98ac17558...|   đường Sông Thao 5|
|Công ty Cổ phần Đ...|   33.540,7m2|   Căn hộ chung cư|Golden City là că...|  25/09/2024 9:05 AM|        TP. Tây Nin

In [31]:
# Xóa bỏ dấu chấm ở các dòng m2
df = df.withColumn("Diện tích:", regexp_replace(col("Diện tích:"), "\\.", ""))

# Đổi dấu phẩy ở dòng có chứa 'ha' thành dấu chấm
df = df.withColumn("Diện tích:", regexp_replace(col("Diện tích:"), ",", "."))

In [32]:
from pyspark.sql.functions import col, when, regexp_extract
# Chuyển đổi giá trị diện tích, nhân với 10.000 nếu có 'ha'
df = df.withColumn(
    "Diện tích:",
    when(
        col("Diện tích:") == "chưa cập nhật",  # Giữ nguyên nếu giá trị là "chưa cập nhật"
        "chưa cập nhật"
    ).otherwise(
        when(
            col("Diện tích:").contains("ha"),
            regexp_extract(col("Diện tích:"), "([\\d.]+)", 1).cast("float") * 10000  # Nhân với 10.000 nếu có 'ha'
        ).otherwise(
            regexp_extract(col("Diện tích:"), "([\\d.]+)", 1).cast("float")  # Chuyển đổi giá trị m2
        )
    )
)
# Đổi tên cột thành "Diện tích(m²)"
df = df.withColumnRenamed("Diện tích:", "Diện tích(m²)")

In [33]:
# Xóa bỏ chữ 'ha' và 'm2'
df = df.withColumn("Diện tích(m²)", regexp_replace(col("Diện tích(m²)"), "ha|m2", ""))

In [34]:
#Xem kiểu dl các cột
df.show()

+--------------------+-------------+------------------+--------------------+--------------------+--------------------+-----------+--------------------+-----------------+--------+--------------------+--------------------+
|          Chủ đầu tư|Diện tích(m²)|        Loại dự án|               Mô tả|           Ngày đăng|           Thành phố| Trạng thái|           Tên dự án|  Tổng vốn đầu tư|Xếp hạng|                 _id|               Đường|
+--------------------+-------------+------------------+--------------------+--------------------+--------------------+-----------+--------------------+-----------------+--------+--------------------+--------------------+
|Công Ty Cổ Phần B...|       8200.0|     Đất Nền Dự Án|The Long Eyes là ...|  12/09/2024 8:49 AM|           Trảng Bom|Đang mở bán|The Long Eyes: Dự...|    chưa cập nhật|     5.0|{66f8f9c98ac17558...|   đường Sông Thao 5|
|Công ty Cổ phần Đ...|      33540.7|   Căn hộ chung cư|Golden City là că...|  25/09/2024 9:05 AM|        TP. Tây Nin

In [35]:
# Xóa bỏ dấu chấm ở các dòng "tỷ đồng"
df = df.withColumn("Tổng vốn đầu tư", regexp_replace(col("Tổng vốn đầu tư"), "\\.", ""))

# Đổi dấu phẩy ở dòng có chứa 'USD' thành dấu chấm
df = df.withColumn("Tổng vốn đầu tư", regexp_replace(col("Tổng vốn đầu tư"), ",", "."))

In [36]:
from pyspark.sql.functions import col, when, regexp_extract
# Chuyển đổi giá trị diện tích, nhân với 10.000 nếu có 'ha'
df = df.withColumn(
    "Tổng vốn đầu tư",
    when(
        col("Tổng vốn đầu tư") == "chưa cập nhật",  # Giữ nguyên nếu giá trị là "chưa cập nhật"
        "chưa cập nhật"
    ).otherwise(
        when(
            col("Tổng vốn đầu tư").contains("USD"),
            regexp_extract(col("Tổng vốn đầu tư"), "([\\d.]+)", 1).cast("float") * 24  # Nhân 24 nếu có chữ triệu USD
        ).otherwise(
            regexp_extract(col("Tổng vốn đầu tư"), "([\\d.]+)", 1).cast("float")  
        )
    )
)
# Đổi tên cột thành "Diện tích(m²)"
df = df.withColumnRenamed("Tổng vốn đầu tư", "Tổng vốn đầu tư(tỷ đồng)")
# Hiển thị dữ liệu sau khi thay đổi
df.show()

+--------------------+-------------+------------------+--------------------+--------------------+--------------------+-----------+--------------------+------------------------+--------+--------------------+--------------------+
|          Chủ đầu tư|Diện tích(m²)|        Loại dự án|               Mô tả|           Ngày đăng|           Thành phố| Trạng thái|           Tên dự án|Tổng vốn đầu tư(tỷ đồng)|Xếp hạng|                 _id|               Đường|
+--------------------+-------------+------------------+--------------------+--------------------+--------------------+-----------+--------------------+------------------------+--------+--------------------+--------------------+
|Công Ty Cổ Phần B...|       8200.0|     Đất Nền Dự Án|The Long Eyes là ...|  12/09/2024 8:49 AM|           Trảng Bom|Đang mở bán|The Long Eyes: Dự...|           chưa cập nhật|     5.0|{66f8f9c98ac17558...|   đường Sông Thao 5|
|Công ty Cổ phần Đ...|      33540.7|   Căn hộ chung cư|Golden City là că...|  25/09/2024

In [37]:
# Thay thế giá trị 0 trong cột 'Xếp hạng' bằng 'chưa có đánh giá'
df = df.withColumn("Xếp hạng", when(col("Xếp hạng") == '0', 'chưa có đánh giá').otherwise(col("Xếp hạng")))
# Hiển thị dữ liệu sau khi thay đổi
df.show()

+--------------------+-------------+------------------+--------------------+--------------------+--------------------+-----------+--------------------+------------------------+----------------+--------------------+--------------------+
|          Chủ đầu tư|Diện tích(m²)|        Loại dự án|               Mô tả|           Ngày đăng|           Thành phố| Trạng thái|           Tên dự án|Tổng vốn đầu tư(tỷ đồng)|        Xếp hạng|                 _id|               Đường|
+--------------------+-------------+------------------+--------------------+--------------------+--------------------+-----------+--------------------+------------------------+----------------+--------------------+--------------------+
|Công Ty Cổ Phần B...|       8200.0|     Đất Nền Dự Án|The Long Eyes là ...|  12/09/2024 8:49 AM|           Trảng Bom|Đang mở bán|The Long Eyes: Dự...|           chưa cập nhật|             5.0|{66f8f9c98ac17558...|   đường Sông Thao 5|
|Công ty Cổ phần Đ...|      33540.7|   Căn hộ chung cư|G

In [38]:
from pyspark.sql.functions import col

# Danh sách cột mong muốn
desired_columns = [
    "_id",  # Thêm cột _id vào đầu
    "Loại dự án",
    "Tên dự án",
    "Đường",
    "Thành phố",
    "Chủ đầu tư",
    "Trạng thái",
    "Diện tích(m²)",  # Đổi tên cột này cho đầy đủ
    "Ngày đăng",
    "Tổng vốn đầu tư(tỷ đồng)",  # Đổi tên cột này cho đầy đủ
    "Xếp hạng",
    "Mô tả"
]

# Kiểm tra danh sách các cột có trong DataFrame
existing_columns = df.columns
print("Các cột hiện có trong DataFrame:", existing_columns)

# Lọc ra các cột tồn tại trong DataFrame
valid_columns = [col for col in desired_columns if col in existing_columns]

# Sắp xếp lại DataFrame theo thứ tự các cột đã lọc
df = df.select(*valid_columns)

# Hiển thị dữ liệu sau khi thay đổi
df.show()


Các cột hiện có trong DataFrame: ['Chủ đầu tư', 'Diện tích(m²)', 'Loại dự án', 'Mô tả', 'Ngày đăng', 'Thành phố', 'Trạng thái', 'Tên dự án', 'Tổng vốn đầu tư(tỷ đồng)', 'Xếp hạng', '_id', 'Đường']
+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+-----------+-------------+--------------------+------------------------+----------------+--------------------+
|                 _id|        Loại dự án|           Tên dự án|               Đường|           Thành phố|          Chủ đầu tư| Trạng thái|Diện tích(m²)|           Ngày đăng|Tổng vốn đầu tư(tỷ đồng)|        Xếp hạng|               Mô tả|
+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+-----------+-------------+--------------------+------------------------+----------------+--------------------+
|{66f8f9c98ac17558...|     Đất Nền Dự Án|The Long Eyes: Dự...|   đường Sông Thao 5|           T

In [40]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, to_json, regexp_extract, when

# 2. Chuyển đổi cột _id từ struct thành string
df = df.withColumn("_id", col("_id").getField("oid"))

# 3. Ghi dữ liệu đã làm sạch vào file CSV
df.write.csv("cleaned_cafeland.csv", header=True, mode="overwrite")



In [41]:
# Dừng SparkSession
spark.stop()